*DELE CA2 Submission Notebook*

# Lunar Lander: Learning to land a rocket! 

|          Name        |      Class    | Admin No. |
|----------------------|---------------|-----------|
| Timothy Chia Kai Lun | DAAA/FT/2B/02 | P2106911  |
|      Lim Jun Jie     | DAAA/FT/2B/02 | P2100788  |

![](https://i.imgur.com/2pzRsfx.jpg)

**<u>Objectives</u>**

We are tasked with training an agent capable of landing the lunar lander safely onto a landing pad [as per documentation](#https://gymnasium.farama.org/environments/box2d/lunar_lander/), using reinforcement learning techniques. The environment is provided to us through the [gymnasium library](#https://gymnasium.farama.org/).

## 1. About The Environment

The Lunar Lander is a very interesting environment as it is described as a classic rocket trajectory optimization problem. The environment comes in two versions: discrete and continuous, with differing action spaces and the goal is to safely land the lunar lander on the launch pad located at (0,0).

For this group assignment, we will be applying different RL algorithms on the discrete version environment which contain 4 actions in the action space and the state space is represented as an 8th dimensional vector.

### 1.1 States and Actions

$$

\text{State Vector}
\begin{cases}
    s_0=\text{x-axis coord of agent} \\
    s_1=\text{y-axis coord of agent} \\
    s_2=\text{x-axis linear velocity} \\
    s_3=\text{y-axis linear velocity} \\
    s_4=\text{Agent's angle} \\
    s_5=\text{Agent's angular velocity} \\
    s_6=\text{Right leg touched ground} \\
    s_7=\text{Left leg touched ground}
\end{cases}

\text{Action Space}
\begin{cases}
    a_0=\text{Do nothing} \\
    a_1=\text{Fire left engine} \\
    a_2=\text{Fire main engine} \\
    a_3=\text{Fire right engine}
\end{cases}

$$

### 1.2 Reward Scheme

- The agent gains 100-140 points for landing on the launch pad and coming to a rest.
- Coming to a rest yields an additional 100 points.
- Each leg with ground contact earns an 10 points.
- Moving away from landing spot decreases the rewards.
- Crashing decreases the rewards by -100 points.
- Firing the main engine decreases rewards by -0.3 points.
- Firing the side engine decreases rewards by -0.3 points.

An episode is considered a solution when the episodic rewards obtained are greater than or equal to 200 points.

## 2. Project Setup

## 3. Reinforcement Learning Algorithms

### 3.1 Baseline: Random Walk

### 3.2 Iterative Q-Learning

### 3.3 Deep Q-Learning (DQN)

### 3.4 State-Action-Reward-State-Action (SARSA)

## 4. Evaluation

## 5. Conclusion